<a href="https://colab.research.google.com/github/moch1996/Vaycold/blob/main/Recommend_Algorithm/%EA%B0%84%EB%8B%A8%ED%95%9C%20%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C%20%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 간단한 추천시스템 만들기
  - ### 목표 : 특정유저가 특정영화에 몇점을 주었는가??
  - 간단한 추천알고리즘 만들기
  - 평점을 예측하고 RMSE로 평가

## 라이브러리 정의

In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
from math import sqrt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.listdir('drive/MyDrive/fastcampus/recommend/movielens')

['ratings.csv', 'tags.csv', 'movies.csv', 'README.txt', 'links.csv']

In [ ]:
path = '/content/drive/MyDrive/fastcampus/recommend/movielens'

In [ ]:
# 데이터 불러오기

ratings_df = pd.read_csv(os.path.join(path,'ratings.csv'), encoding='utf-8')
movies_df = pd.read_csv(os.path.join(path,'movies.csv'), index_col = 'movieId', encoding='utf-8')
tags_df = pd.read_csv(os.path.join(path,'tags.csv'), encoding='utf-8')

In [ ]:
print(ratings_df.shape)
print(ratings_df.head())

(100836, 4)
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [ ]:
# Ratings data 확인